In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rossmann-store-sales/store.csv
/kaggle/input/rossmann-store-sales/train.csv
/kaggle/input/rossmann-store-sales/test.csv


In [2]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [3]:
from fastai.tabular.all import *
from pandas.api.types import is_string_dtype , is_numeric_dtype, is_categorical_dtype
from sklearn.tree import DecisionTreeRegressor

In [4]:
!pip install dtreeviz

In [5]:
from dtreeviz.trees import *
import dtreeviz

In [6]:
train_df = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv', low_memory=False)
test_df =pd.read_csv('/kaggle/input/rossmann-store-sales/test.csv', low_memory=False)
store_df = pd.read_csv('/kaggle/input/rossmann-store-sales/store.csv', low_memory=False)

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [8]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [9]:
df = train_df.merge(store_df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

In [11]:
df = add_datepart(df, 'Date')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 30 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Sales                      1017209 non-null  int64  
 3   Customers                  1017209 non-null  int64  
 4   Open                       1017209 non-null  int64  
 5   Promo                      1017209 non-null  int64  
 6   StateHoliday               1017209 non-null  object 
 7   SchoolHoliday              1017209 non-null  int64  
 8   StoreType                  1017209 non-null  object 
 9   Assortment                 1017209 non-null  object 
 10  CompetitionDistance        1014567 non-null  float64
 11  CompetitionOpenSinceMonth  693861 non-null   float64
 12  CompetitionOpenSinceYear   693861 non-null   float64
 13  Promo2      

In [13]:
df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1,5,5263,555,1,1,0,1,c,a,...,31,4,212,True,False,False,False,False,False,1.438301e+09
1,2,5,6064,625,1,1,0,1,a,a,...,31,4,212,True,False,False,False,False,False,1.438301e+09
2,3,5,8314,821,1,1,0,1,a,a,...,31,4,212,True,False,False,False,False,False,1.438301e+09
3,4,5,13995,1498,1,1,0,1,c,c,...,31,4,212,True,False,False,False,False,False,1.438301e+09
4,5,5,4822,559,1,1,0,1,a,a,...,31,4,212,True,False,False,False,False,False,1.438301e+09


In [14]:
dep_var = 'Sales'
df[dep_var] = np.log(df[dep_var] + 1)

In [15]:
proc = [Categorify, FillMissing]

In [16]:
cond = (df.Year < 2014) | (df.Month < 12)
train_idx = np.where(cond)[0]
valid_idx = np.where(~cond)[0]

splits = list(train_idx), list(valid_idx)

In [17]:
cont, cat = cont_cat_split(df, 1, dep_var=dep_var) 

In [18]:
to = TabularPandas(df, proc, cat, cont, y_names=dep_var, splits=splits)

In [19]:
len(to.train), len(to.valid)

(988224, 28985)

In [20]:
to.show(5)

,StateHoliday,StoreType,Assortment,PromoInterval,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,CompetitionDistance_na,CompetitionOpenSinceMonth_na,CompetitionOpenSinceYear_na,Promo2SinceWeek_na,Promo2SinceYear_na,Store,DayOfWeek,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Week,Day,Dayofweek,Dayofyear,Elapsed,Sales
0,0,c,a,#na#,True,False,False,False,False,False,False,False,False,True,True,1,5,555,1,1,1,1270.0,9.0,2008.0,0,22.0,2012.0,2015,7,31,31,4,212,1.438301e+09,8.568646
1,0,a,a,"Jan,Apr,Jul,Oct",True,False,False,False,False,False,False,False,False,False,False,2,5,625,1,1,1,570.0,11.0,2007.0,1,13.0,2010.0,2015,7,31,31,4,212,1.438301e+09,8.710290
2,0,a,a,"Jan,Apr,Jul,Oct",True,False,False,False,False,False,False,False,False,False,False,3,5,821,1,1,1,14130.0,12.0,2006.0,1,14.0,2011.0,2015,7,31,31,4,212,1.438301e+09,9.025816
3,0,c,c,#na#,True,False,False,False,False,False,False,False,False,True,True,4,5,1498,1,1,1,620.0,9.0,2009.0,0,22.0,2012.0,2015,7,31,31,4,212,1.438301e+09,9.546527
4,0,a,a,#na#,True,False,False,False,False,False,False,False,False,True,True,5,5,559,1,1,1,29910.0,4.0,2015.0,0,22.0,2012.0,2015,7,31,31,4,212,1.438301e+09,8.481152


In [21]:
to.items.head(5)

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,CompetitionDistance_na,CompetitionOpenSinceMonth_na,CompetitionOpenSinceYear_na,Promo2SinceWeek_na,Promo2SinceYear_na
0,1,5,8.568646,555,1,1,1,1,3,1,...,1,1,1,1,1.438301e+09,1,1,1,2,2
1,2,5,8.710290,625,1,1,1,1,1,1,...,1,1,1,1,1.438301e+09,1,1,1,1,1
2,3,5,9.025816,821,1,1,1,1,1,1,...,1,1,1,1,1.438301e+09,1,1,1,1,1
3,4,5,9.546527,1498,1,1,1,1,3,3,...,1,1,1,1,1.438301e+09,1,1,1,2,2
4,5,5,8.481152,559,1,1,1,1,1,1,...,1,1,1,1,1.438301e+09,1,1,1,2,2


In [22]:
save_pickle('to.pkl', to)

In [23]:
to = load_pickle('to.pkl')

In [24]:
xs, y = to.train.xs, to.train.y
valid_xs, valid_y = to.valid.xs, to.valid.y

In [25]:
m = DecisionTreeRegressor()
m.fit(xs, y);

In [26]:
def rmse(preds, y): return round(math.sqrt(((preds-y)**2).mean()) ,6)
def m_rmse(m, xs, y): return rmse(m.predict(xs), y)

In [27]:
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

(0.0, 0.112603)

In [28]:
m.get_n_leaves(), len(xs)

(818526, 988224)